In [14]:
import os
import re
import time
import pandas as pd

from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

In [15]:
path = "./chromedriver"

In [30]:
browser = Chrome(path)
browser.implicitly_wait(5)
#browser.maximize_window()
browser.get("https://www.youtube.com/watch?v=VKbueci1j3U")
pagedown(3, time_setting=6)
#pagedown_to_bottom()

In [17]:
def pagedown(page_down_num=1, time_setting=3):
    '''
    page_down_num: 페이지 다운 클릭 횟수
    time_setting: 빨리 눌러서 씹힘 방지 웨이팅 시간(초)
    '''
    body = browser.find_element_by_tag_name('body')
    while page_down_num:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(time_setting)
        page_down_num -= 1

In [18]:
pagedown(10, time_setting=6)

In [22]:
def pagedown_to_bottom(time_setting=5):
    '''
    time_setting: 빨리 눌러서 씹힘 방지 웨이팅 시간(초)
    '''
    time.sleep(time_setting)
    scroll_pause_time = time_setting
    last_height = browser.execute_script(
        "return document.documentElement.scrollHeight"
    )
    while True:
        # 스크롤 끝까지 내리기
        browser.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")
        # 로딩 기다리기 // 넘 빨리 돌리면 씹힘
        time.sleep(scroll_pause_time)
        # 새로운 높이와 갱신된 높이를 비교해서 같으면 종료
        new_height = browser.execute_script(
            "return document.documentElement.scrollHeight"
        )
        if new_height == last_height:
            break
        last_height = new_height

In [23]:
pagedown_to_bottom()

In [41]:
def youtube_comment_crawler():
    '''
    댓글 크롤링 시작
    ## 댓글의 댓글은 크롤링 안됩 ##
    '''
    # 데이터 프레임 안에 저장
    comment_data = pd.DataFrame({
        "youtube_ID" : [],
        "cmt" : [],
        "like_num" : []
    })
    # html source
    html = browser.page_source
    src = BeautifulSoup(html, 'html.parser' )
    
    # 댓글 찾기 -> html tag 와 class로 찾음
    comment = src.find_all(
    "ytd-comment-renderer", attrs={'class':"style-scope ytd-comment-thread-renderer"}
    )
    for i in range(len(comment)):
        comment0 =comment[i].find('yt-formatted-string', {'id': 'content-text', 'class': 'style-scope ytd-comment-renderer'}).text
        
        # 좋아요
        try:
            aa = comment[i].find('span', {'id': 'vote-count-left'}).text
            like_num = "".join(re.findall("[0-9]", aa)) + "개"
        except:
            like_num = 0
        # ID
        bb = comment[i].find('a', {'id': 'author-text'}).find('span').text
        youtube_id = "".join(re.findall("[가-힣0-9A-Za-z]", bb))
        
        insert_data = pd.DataFrame({
            "youtube_ID" : [youtube_id],
            "cmt" : [comment0],
            "like_num" : [like_num]
            })
        comment_data = comment_data.append(insert_data)
    comment_data = comment_data.reset_index(drop=True)
    return comment_data

In [33]:
html = browser.page_source

In [34]:
html[:100]

'<html style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" lang="ko-KR" typography="" sta'

In [35]:
src = BeautifulSoup(html, 'html.parser')

In [36]:
comment = src.find_all(
"ytd-comment-renderer", attrs={'class':"style-scope ytd-comment-thread-renderer"}
)

In [37]:
comment

[<ytd-comment-renderer class="style-scope ytd-comment-thread-renderer" comment-style="unknown" id="comment"><!--css-build:shady--><div class="style-scope ytd-comment-renderer" id="body">
 <div class="style-scope ytd-comment-renderer" id="author-thumbnail">
 <a class="yt-simple-endpoint style-scope ytd-comment-renderer" href="/channel/UCBuJa6Q9vhP6nMu0uGffhJQ">
 <yt-img-shadow class="style-scope ytd-comment-renderer no-transition" fit="" height="40" loaded="" style="background-color: transparent;" width="40"><!--css-build:shady--><img alt="john오민석" class="style-scope yt-img-shadow" height="40" id="img" src="https://yt3.ggpht.com/ytc/AAUvwngpbM1GjqQuAPxzr4KrpSJnmwkR6xwdq_OIeA=s48-c-k-c0x00ffffff-no-rj" width="40"/></yt-img-shadow>
 </a>
 </div>
 <div class="style-scope ytd-comment-renderer" id="main">
 <div class="style-scope ytd-comment-renderer" id="header">
 <div class="style-scope ytd-comment-renderer" hidden="" id="header-badge">
 <div class="style-scope ytd-comment-renderer" hidden

In [42]:
len(comment)

30

In [43]:
data = youtube_comment_crawler()

In [44]:
data['like_num'] = data['like_num'].map(lambda x : x[:-1])

In [46]:
data['like_num'] = data['like_num'].astype(int)

In [47]:
data.sort_values(by="like_num", ascending=False)

,youtube_ID,cmt,like_num
13,BonyE,제발 이번 콘서트 딴 건 몰라도 마이스토리는 무조건 불러주셔야 합니다 행님,13
11,eunyoungjo,보고싶따 유형 💚,8
4,호호커플,나얼님이 진짜 날아오르시는줄 알았잖아요.... 영광스러웠습니다!!,7
1,김민규,언제부턴지 모를 들려오는 고요한 바람이\n마치 익숙한 듯 난 귀를 기울여 가까이 다...,7
0,john오민석,나얼영상은 폰이 다 울리네....ㄷㄷ레전드여ㅎㅎ,6
3,율빡6982,이런 ccm스타일의 곡도 더많이 내주셨슴좋겠습니다^^,6
26,응응,50살까지 몇년 안남았다,5
5,Epic,은혜롭습니다... 매우 감사.,4
7,소우리,지친 하루에 위안이 되네요..ㅜㅜ,4
25,재순이누나,저 형 성대에 예수 사는 거 같애,4


In [48]:
data

,youtube_ID,cmt,like_num
0,john오민석,나얼영상은 폰이 다 울리네....ㄷㄷ레전드여ㅎㅎ,6
1,김민규,언제부턴지 모를 들려오는 고요한 바람이\n마치 익숙한 듯 난 귀를 기울여 가까이 다...,7
2,JameyYoo,아멘 🙏🙏🙏,3
3,율빡6982,이런 ccm스타일의 곡도 더많이 내주셨슴좋겠습니다^^,6
4,호호커플,나얼님이 진짜 날아오르시는줄 알았잖아요.... 영광스러웠습니다!!,7
5,Epic,은혜롭습니다... 매우 감사.,4
6,장건우,아멘,0
7,소우리,지친 하루에 위안이 되네요..ㅜㅜ,4
8,율빡6982,"이런 분위기의 코드진행과 템포,스타일을 나얼님께서 불러주시는 곡이 많았으면 좋겠서요~^^",1
9,PatrickKim,4:43을듣고 지렸습니다,1


In [51]:
def youtube_video_url_crawler(limit_num=10):
    '''특정 유튜버의 동영상 카테고리의  url이어야 함'''
    html = browser.page_source
    src = BeautifulSoup(html, 'html.parser')
    
    video_url = src.find_all("a", attrs={
        'id': 'thumbnail', 'class': 'yt-simple-endpoint inline-block style-scope ytd-thumbnail'
    })
    urls = []
    length = 0
    for row in video_url:
        if row.get('href') != None:
            address = row.get("href")
            true_address = 'https://www.youtube.com' + address
            urls.append(true_address)
            length += 1
        if length >= limit_num:
            break
    return urls

In [53]:
browser = Chrome(path)
browser.implicitly_wait(5)
browser.get("https://www.youtube.com/channel/UChhNzuTuJeWDeJFzDeW0hJw/videos")
pagedown(5, time_setting=6)
url_list = youtube_video_url_crawler(limit_num=20)

In [54]:
url_list

['https://www.youtube.com/watch?v=FRRyR8QDoLs',
 'https://www.youtube.com/watch?v=VaE1r-wdG_c',
 'https://www.youtube.com/watch?v=vvpOdRTqqjA',
 'https://www.youtube.com/watch?v=yCMooI1_Ffc',
 'https://www.youtube.com/watch?v=i3xwR3yMgOg',
 'https://www.youtube.com/watch?v=tXvCneu0B6U',
 'https://www.youtube.com/watch?v=FTDiCdxV-HY',
 'https://www.youtube.com/watch?v=lk6qP5XEyVU',
 'https://www.youtube.com/watch?v=2WS3yClWNFQ',
 'https://www.youtube.com/watch?v=-a9WIOK92-w',
 'https://www.youtube.com/watch?v=ic3CR6KLeA4',
 'https://www.youtube.com/watch?v=GtvjEH9miN4',
 'https://www.youtube.com/watch?v=wyPah9dFfvk',
 'https://www.youtube.com/watch?v=QMZj5H-HLrI',
 'https://www.youtube.com/watch?v=2z_KNyDWVQc',
 'https://www.youtube.com/watch?v=9fgsBgrLwe4',
 'https://www.youtube.com/watch?v=9shCOIPQxPs',
 'https://www.youtube.com/watch?v=s0E_G9iz9F4',
 'https://www.youtube.com/watch?v=UnYm7cO0p5w',
 'https://www.youtube.com/watch?v=cl3NXN7Hqzk']

In [55]:
browser = Chrome(path)
browser.implicitly_wait(5)
for u in url_list:
    time.sleep(3)
    browser.get(u)